In [1]:
from sklearn.model_selection import cross_val_score
from data import reducir_train, reducir_test, write_submission
%matplotlib inline

In [2]:
train = reducir_train()

In [3]:
train.head()

,expenses,floor,price_aprox_usd,price_usd_per_m2,rooms,surface_covered_in_m2,surface_total_in_m2
0,0.0,0.0,120000.0,360.360360,0.0,93.0,333.0
1,0.0,0.0,220000.0,1000.000000,0.0,220.0,220.0
4,0.0,0.0,185000.0,616.666667,5.0,103.0,300.0
5,0.0,0.0,97000.0,0.000000,2.0,54.0,0.0
6,0.0,0.0,750000.0,0.000000,1.0,0.0,0.0


In [4]:
train_label = []
train_data = []

train_label = train.iloc[:, 2]
train_data = train.iloc[:, train.columns != 'price_aprox_usd']
print("train_data set has {0[0]} rows and {0[1]} columns".format(train_data.shape))

train_data set has 196439 rows and 6 columns


In [5]:
#-----------------------------------------------------------------
# Modelo Random Forest Regressor
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor

In [16]:
modeloRFR=RandomForestRegressor() #Por defecto (n_estimators=10) da mejor que con n=4 y n=30

#modeloRFR.fit(train_data,train_label)

In [14]:
precision = cross_val_score(modeloRFR, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 36240088419.18477


In [17]:
test = reducir_test()

In [18]:
modeloRFR.fit(train_data, train["price_aprox_usd"])

result = modeloRFR.predict(test)

In [28]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/RFR_n4.csv")

In [19]:
del modeloRFR

In [91]:
#-----------------------------------------------------------------
# Modelo KNN
#-----------------------------------------------------------------
from sklearn.neighbors import KNeighborsRegressor

In [98]:
modeloKNN=KNeighborsRegressor(n_neighbors=3) #Mejor que por defecto (n_neighbors=5) y n=10

#modeloKNN.fit(train_data,train_label)

In [99]:
precision = cross_val_score(modeloKNN, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 53813852220.62436


In [97]:
modeloKNN.fit(train_data, train["price_aprox_usd"])

result = modeloKNN.predict(test)

In [45]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/KNN_n4.csv")

In [46]:
del modeloKNN

In [20]:
#-----------------------------------------------------------------
# Modelo de Regresión Lineal
#-----------------------------------------------------------------
from sklearn.linear_model import LinearRegression

In [21]:
modeloLR=LinearRegression()

#modeloLR.fit(train_data,train_label)

In [23]:
precision = cross_val_score(modeloLR, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 153792306661.79938


In [58]:
modeloLR.fit(train_data, train["price_aprox_usd"])

result = modeloLR.predict(test)

In [60]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/LR.csv")

In [24]:
del modeloLR

In [25]:
#-----------------------------------------------------------------
# Modelo de Lasso
#-----------------------------------------------------------------
from sklearn.linear_model import Lasso

In [26]:
modeloLasso=Lasso()

#modeloLasso.fit(train_data,train_label)

In [28]:
precision = cross_val_score(modeloLasso, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 153792316052.93683


In [75]:
modeloLasso.fit(train_data, train["price_aprox_usd"])

result = modeloLasso.predict(test)

In [77]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/Lasso.csv")

In [29]:
del modeloLasso

In [30]:
#-----------------------------------------------------------------
# Modelo Ridge
#-----------------------------------------------------------------
from sklearn.linear_model import Ridge

In [31]:
modeloRidge=Ridge()

#modeloRidge.fit(train_data,train_label)

In [33]:
precision = cross_val_score(modeloRidge, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 153792307434.26871


In [88]:
modeloRidge.fit(train_data, train["price_aprox_usd"])

result = modeloRidge.predict(test)

In [90]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/Ridge.csv")

In [34]:
del modeloRidge

In [35]:
#-----------------------------------------------------------------
# Modelo GradientBoostingRegressor
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor

In [43]:
modeloGBR=GradientBoostingRegressor(n_estimators=150) #Mejor que por defecto (n=100) y n=50. Queda probar con numeros mas
#grandes

#modeloGBR.fit(train_data,train_label)

In [44]:
precision = cross_val_score(modeloGBR, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 33458932957.58320


In [31]:
modeloGBR.fit(train_data, train["price_aprox_usd"])

result = modeloGBR.predict(test)

In [33]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/GBR_n200.csv")

In [45]:
del modeloGBR

In [46]:
#-----------------------------------------------------------------
# Modelo BaggingRegressor
#-----------------------------------------------------------------
from sklearn.ensemble import BaggingRegressor

In [57]:
modeloBR=BaggingRegressor(n_estimators=5) #Mejor que por defecto (n=10) y n=30

#modeloBR.fit(train_data,train_label)

In [58]:
precision = cross_val_score(modeloBR, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 41196238784.53815


In [ ]:
modeloBR.fit(train_data, train["price_aprox_usd"])

result = modeloBR.predict(test)

In [ ]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/BR.csv")

In [59]:
del modeloBR

In [72]:
#-----------------------------------------------------------------
# Modelo AdaBoostRegressor
#-----------------------------------------------------------------
from sklearn.ensemble import AdaBoostRegressor

In [79]:
modeloABR=AdaBoostRegressor() #Por defecto (n_estimators=50) es mejor que n=10 y n=100

#modeloABR.fit(train_data,train_label)

In [80]:
precision = cross_val_score(modeloABR, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 134325890374.98872


In [ ]:
modeloABR.fit(train_data, train["price_aprox_usd"])

result = modeloABR.predict(test)

In [ ]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/ABR.csv")

In [84]:
del modeloABR

In [60]:
#-----------------------------------------------------------------
# Modelo BayesianRidge
#-----------------------------------------------------------------
from sklearn.linear_model import BayesianRidge

In [61]:
modeloBRidge=BayesianRidge()

#modeloBRidge.fit(train_data,train_label)

In [63]:
precision = cross_val_score(modeloBRidge, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 153795400495.94611


In [ ]:
modeloBRidge.fit(train_data, train["price_aprox_usd"])

result = modeloBRidge.predict(test)

In [ ]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/BRidge.csv")

In [64]:
del modeloBRidge

In [65]:
#-----------------------------------------------------------------
# Modelo ElasticNet
#-----------------------------------------------------------------
from sklearn.linear_model import ElasticNet

In [82]:
modeloElasticNet=ElasticNet()

#modeloElasticNet.fit(train_data,train_label)

In [70]:
precision = cross_val_score(modeloElasticNet, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 153865580557.19659


In [ ]:
modeloElasticNet.fit(train_data, train["price_aprox_usd"])

result = modeloElasticNet.predict(test)

In [ ]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/ElasticNet.csv")

In [83]:
del modeloElasticNet

In [85]:
#-----------------------------------------------------------------
# Modelo DecisionTreeRegressor
#-----------------------------------------------------------------
from sklearn.tree import DecisionTreeRegressor

In [86]:
modeloDecisionTreeR=DecisionTreeRegressor()

#modeloDecisionTreeR.fit(train_data,train_label)

In [88]:
precision = cross_val_score(modeloDecisionTreeR, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 44023541579.36916


In [89]:
modeloDecisionTreeR.fit(train_data, train["price_aprox_usd"])

result = modeloDecisionTreeR.predict(test)

In [90]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/DecisionTreeRV2.csv")